In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [162]:
df = pd.read_csv("../missing_data.csv")

In [4]:
df.head()

,Error Encountered,_id
0,Charity name: Église de Jésus-Christ: «Le bon...,6349542e044a88e546303b7d
1,Charity name: A & O: Support Services for Old...,6349559a6f4326e0a00a61bf
2,Charity name: A Way Home: Working Together to...,634955aa6f4326e0a00a61ff
3,Charity name: Acumen Canada BN/Registration ...,634955d46f4326e0a00a62c0
4,Charity name: Affordable Housing: Vancouver I...,634955de6f4326e0a00a62fa


In [11]:
for text in df['Error Encountered']:
    print(text)
    print("\n")
    print(len(text.split("  ")))
    print("\n\n")

Charity name:  Église de Jésus-Christ: «Le bon berger»  BN/Registration number:  837855501RR0001  Charity status: Registered  Effective date of status: 2011-12-16  Sanction: Note: this link will load in another window or tab  Address: 937 RUE ROUVILLE  City: REPENTIGNY  Province, territory, outside of Canada:  QUEBEC  Postal code/Zip code: J5Y1X9  Charity type:  Advancement of Religion  Category:  Christianity


16



Charity name:  A & O: Support Services for Older Adults Inc.  BN/Registration number:  106688377RR0001  Charity status: Registered  Effective date of status: 1969-01-01  Sanction: Note: this link will load in another window or tab  Address: 200-207 DONALD STREET  City: WINNIPEG  Province, territory, outside of Canada:  MANITOBA  Postal code/Zip code: R3C1M5  Charity type:  Relief of Poverty  Category:  Organizations Relieving Poverty


16



Charity name:  A Way Home: Working Together to End Youth Homelessness  BN/Registration number:  795393891RR0001  Charity status: Reg

In [167]:
def parse_charity_details(detailed_text):
    details = {}
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab", "")
        .split(": ")
    )
    print(formatted)
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [168]:
def parse_errors(text):
    formatted = text.split("BN/Registration number:")
    reformat = formatted[0].split(":  ")
    toreformat = "BN/Registration number:" + formatted[1]
    details = parse_charity_details(toreformat)
    if details:
        details["Charity name:"] = reformat[1]
    return details
    

In [169]:
reformatted = []
errors = []
for text in df['Error Encountered']:
    details = parse_errors(text)
    if details:
        reformatted.append(details)
    else:
        errors.append(text)

['BN/Registration number:', '837855501RR0001', 'Charity status', 'Registered', 'Effective date of status', '2011-12-16', '', 'Address', '937 RUE ROUVILLE', 'City', 'REPENTIGNY', 'Province, territory, outside of Canada:', 'QUEBEC', 'Postal code/Zip code', 'J5Y1X9', 'Charity type:', 'Advancement of Religion', 'Category:', 'Christianity']
['BN/Registration number:', '106688377RR0001', 'Charity status', 'Registered', 'Effective date of status', '1969-01-01', '', 'Address', '200-207 DONALD STREET', 'City', 'WINNIPEG', 'Province, territory, outside of Canada:', 'MANITOBA', 'Postal code/Zip code', 'R3C1M5', 'Charity type:', 'Relief of Poverty', 'Category:', 'Organizations Relieving Poverty']
['BN/Registration number:', '795393891RR0001', 'Charity status', 'Registered', 'Effective date of status', '2022-01-01', '', 'Address', '720 BATHURST ST', 'City', 'TORONTO', 'Province, territory, outside of Canada:', 'ONTARIO', 'Postal code/Zip code', 'M5S2R4', 'Charity type:', 'Other purposes beneficial 

In [171]:
len(reformatted)

202

In [172]:
with open("first_round_error_handling.json", "w+") as f:
    json.dump(reformatted, f, indent=3)

In [80]:
len(errors)

58

In [78]:
outside_canada = []
other_errors = []
for error in errors:
    if "Province, territory, outside of Canada:  Postal code/Zip code:" in error:
        #print(error.split("Charity type:")[0][-10:])
        outside_canada.append(error)
    else:
        other_errors.append(error)

In [77]:
len(outside_canada)

25

In [100]:
def parse_outside_canada_charity_details(detailed_text):
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab", "")
        .replace("Province, territory, outside of Canada:", "Province, territory, outside of Canada: Outside of Canada")
        .split(": ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [101]:
outside_canada_formatted = []
for text in outside_canada:
    formatted_text = parse_outside_canada_charity_details(text)
    if formatted_text:
        outside_canada_formatted.append(formatted_text)
    else:
        print(text)

In [102]:
outside_canada_formatted

[{'Charity name:': 'Acumen Canada',
  'BN/Registration number:': '818192130RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '2013-07-16',
  'Address': '40 WORTH STREET',
  'City': 'NEW YORK',
  'Province, territory, outside of Canada': 'Outside of Canada',
  'Postal code/Zip code': '10013',
  'Charity type:': 'Relief of Poverty',
  'Category:': 'Organizations Relieving Poverty'},
 {'Charity name:': 'BLUE SKIES CULTURAL CENTRE',
  'BN/Registration number:': '130206220RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '1994-01-01',
  'Address': '5141 CHESSIE CIRCLE',
  'City': 'HALTOM CITY',
  'Province, territory, outside of Canada': 'Outside of Canada',
  'Postal code/Zip code': '76137',
  'Charity type:': 'Other purposes beneficial to the community',
  'Category:': 'Arts'},
 {'Charity name:': 'Canadian Stress Society Stresshelp /La Société Canadienne du Stress Stressaide',
  'BN/Registration number:': '816792295RR0001',
  'Charity status':

In [103]:
with open("outside_canada_errors_fixed.json", "w+") as f:
    json.dump(outside_canada_formatted, f, indent=3)

In [79]:
len(other_errors)

33

In [85]:
attention_people = []
more_errors = []
for error in other_errors:
    if "ATT" in error:
        attention_people.append(error)
    else:
        more_errors.append(error)

In [84]:
len(attention_people)

14

In [106]:
def parse_attention_people_charity_details(detailed_text):
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab", "")
        .replace("ATT:", "{[ATTN]}")
        .replace("ATTN:", "{[ATTN]}")
        .replace("ATTENTION:", "{[ATTN]}")
        .split(": ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [107]:
attention_people_formatted = []
for text in attention_people:
    formatted_text = parse_attention_people_charity_details(text)
    if formatted_text:
        attention_people_formatted.append(formatted_text)
    else:
        print(text)

In [108]:
attention_people_formatted

[{'Charity name:': 'BENEVOLENT SOCIETY "HEIDEHOF" FOR THE CARE OF THE AGED',
  'BN/Registration number:': '107474199RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '1967-01-01',
  'Address': '{[ATTN]} DIRECTOR OF FINANCE',
  'City': 'ST. CATHARINES',
  'Province, territory, outside of Canada:': 'ONTARIO',
  'Postal code/Zip code': 'L2N4J4',
  'Charity type:': 'Relief of Poverty',
  'Category:': 'Organizations Relieving Poverty'},
 {'Charity name:': 'BLOOMFIELD UNITED CHURCH',
  'BN/Registration number:': '106798390RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '1967-01-01',
  'Address': '{[ATTN]} TREASURER',
  'City': 'BLOOMFIELD',
  'Province, territory, outside of Canada:': 'ONTARIO',
  'Postal code/Zip code': 'K0K1G0',
  'Charity type:': 'Advancement of Religion',
  'Category:': 'Christianity'},
 {'Charity name:': 'CANSO UNITED BAPTIST CHURCH',
  'BN/Registration number:': '118837368RR0001',
  'Charity status': 'Registered',
  'Effe

In [173]:
with open("attention_people_errors_fixed.json", "w+") as f:
    json.dump(attention_people_formatted, f, indent=3)

In [92]:
po_boxes = []
no_address = []
even_more = []
for error in more_errors:
    if "P. O. BOX:" in error:
        po_boxes.append(error)
    elif "Address: City:" in error:
        no_address.append(error)
    else:
        even_more.append(error)

In [110]:
def parse_po_box_charity_details(detailed_text):
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab", "")
        .replace("P. O. BOX:", "{[P. O. BOX]}")
        .split(": ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [111]:
po_box_charity = parse_po_box_charity_details(po_boxes[0])

In [112]:
po_box_charity

{'Charity name:': 'CAMP GITCHIGOMEE',
 'BN/Registration number:': '829098813RR0001',
 'Charity status': 'Registered',
 'Effective date of status': '1980-08-23',
 'Address': '{[P. O. BOX]} 29018',
 'City': 'THUNDER BAY',
 'Province, territory, outside of Canada:': 'ONTARIO',
 'Postal code/Zip code': 'P7B6P9',
 'Charity type:': 'Other purposes beneficial to the community',
 'Category:': 'Public Amenities'}

In [93]:
po_boxes

['Charity name:  CAMP GITCHIGOMEE  BN/Registration number:  829098813RR0001  Charity status: Registered  Effective date of status: 1980-08-23  Sanction: Note: this link will load in another window or tab  Address: P. O. BOX: 29018  City: THUNDER BAY  Province, territory, outside of Canada:  ONTARIO  Postal code/Zip code: P7B6P9  Charity type:  Other purposes beneficial to the community  Category:  Public Amenities']

In [128]:
def parse_no_address_charity_details(detailed_text):
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab", "")
        .replace("Address: City:", "Address: {[None]}: City:")
        .split(": ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [129]:
no_address_formatted = []
for text in no_address:
    formatted_text = parse_no_address_charity_details(text)
    if formatted_text:
        no_address_formatted.append(formatted_text)
    else:
        print("\n")

In [130]:
no_address_formatted

[{'Charity name:': 'FABRIQUE DE LA PAROISSE DE SAINT-OURS',
  'BN/Registration number:': '118906429RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '1967-01-01',
  'Address': '{[None]}',
  'City': 'ST-OURS SUR RICHELIEU',
  'Province, territory, outside of Canada:': 'QUEBEC',
  'Postal code/Zip code': 'J0G1P0',
  'Charity type:': 'Advancement of Religion',
  'Category:': 'Christianity'},
 {'Charity name:': 'GOSPEL LIGHT PENTECOSTAL TABERNACLE',
  'BN/Registration number:': '107833394RR0031',
  'Charity status': 'Registered',
  'Effective date of status': '1976-10-19',
  'Address': '{[None]}',
  'City': "COBB'S ARM",
  'Province, territory, outside of Canada:': 'NEWFOUNDLAND AND LABRADOR',
  'Postal code/Zip code': 'A0G2R0',
  'Charity type:': 'Advancement of Religion',
  'Category:': 'Christianity'},
 {'Charity name:': 'LAUDER UNITED CHURCH BOARD',
  'BN/Registration number:': '119009520RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '19

In [132]:
no_address_formatted.append(po_box_charity)

In [140]:
with open("no_address_errors_fixed.json", "w+") as f:
    json.dump(no_address_formatted, f, indent=3)

In [94]:
no_address

['Charity name:  FABRIQUE DE LA PAROISSE DE SAINT-OURS  BN/Registration number:  118906429RR0001  Charity status: Registered  Effective date of status: 1967-01-01  Sanction: Note: this link will load in another window or tab  Address: City: ST-OURS SUR RICHELIEU  Province, territory, outside of Canada:  QUEBEC  Postal code/Zip code: J0G1P0  Charity type:  Advancement of Religion  Category:  Christianity',
 "Charity name:  GOSPEL LIGHT PENTECOSTAL TABERNACLE  BN/Registration number:  107833394RR0031  Charity status: Registered  Effective date of status: 1976-10-19  Sanction: Note: this link will load in another window or tab  Address: City: COBB'S ARM  Province, territory, outside of Canada:  NEWFOUNDLAND AND LABRADOR  Postal code/Zip code: A0G2R0  Charity type:  Advancement of Religion  Category:  Christianity",
 'Charity name:  LAUDER UNITED CHURCH BOARD  BN/Registration number:  119009520RR0001  Charity status: Registered  Effective date of status: 1967-01-01  Sanction: Note: this li

In [134]:
def parse_po_box2_charity_details(detailed_text):
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab", "")
        .replace("PO. BOX :", "{[P. O. BOX]}")
        .split(": ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [136]:
po_box_2 = parse_po_box2_charity_details(even_more[3])

In [139]:
no_address_formatted.append(po_box_2)

In [142]:
penalized_charities = []
for text in even_more:
    if "Penalized" in text:
        penalized_charities.append(text)

In [143]:
penalized_charities

['Charity name:  Universal Educational Institute of Canada  BN/Registration number:  139482715RR0001  Charity status: Registered  Effective date of status: 1993-01-01  Sanction: Note: this link will load in another window or tab  Penalized  Address: 3000-8882 170 ST  City: EDMONTON  Province, territory, outside of Canada:  ALBERTA  Postal code/Zip code: T5T4M2  Charity type:  Advancement of Education  Category:  Teaching Institutions',
 'Charity name:  Universal Educational Institute of Canada  BN/Registration number:  139482715RR0001  Charity status: Registered  Effective date of status: 1993-01-01  Sanction: Note: this link will load in another window or tab  Penalized  Address: 3000-8882 170 ST  City: EDMONTON  Province, territory, outside of Canada:  ALBERTA  Postal code/Zip code: T5T4M2  Charity type:  Advancement of Education  Category:  Teaching Institutions',
 "Charity name:  VANCOUVER FIRE FIGHTERS' CHARITABLE SOCIETY  BN/Registration number:  867412132RR0001  Charity status: 

In [152]:
def parse_penalized_charity_details(detailed_text):
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab: Penalized", "Sanction: Penalized")
        .split(": ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [153]:
penalized_formatted = []
for text in penalized_charities:
    formatted = parse_penalized_charity_details(text)
    if formatted:
        penalized_formatted.append(formatted)
    else:
        print(text)

In [154]:
penalized_formatted

[{'Charity name:': 'Universal Educational Institute of Canada',
  'BN/Registration number:': '139482715RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '1993-01-01',
  'Sanction': 'Penalized',
  'Address': '3000-8882 170 ST',
  'City': 'EDMONTON',
  'Province, territory, outside of Canada:': 'ALBERTA',
  'Postal code/Zip code': 'T5T4M2',
  'Charity type:': 'Advancement of Education',
  'Category:': 'Teaching Institutions'},
 {'Charity name:': 'Universal Educational Institute of Canada',
  'BN/Registration number:': '139482715RR0001',
  'Charity status': 'Registered',
  'Effective date of status': '1993-01-01',
  'Sanction': 'Penalized',
  'Address': '3000-8882 170 ST',
  'City': 'EDMONTON',
  'Province, territory, outside of Canada:': 'ALBERTA',
  'Postal code/Zip code': 'T5T4M2',
  'Charity type:': 'Advancement of Education',
  'Category:': 'Teaching Institutions'},
 {'Charity name:': "VANCOUVER FIRE FIGHTERS' CHARITABLE SOCIETY",
  'BN/Registration number:': '

In [155]:
with open("penalized_errors_fixed.json", "w+") as f:
    json.dump(penalized_formatted, f, indent=3)

In [65]:
def parse_us_charity_details(detailed_text):
    details = {}
    formatted = (
        detailed_text.replace("  ", ": ")
        .replace("Sanction: Note: this link will load in another window or tab", "")
        .replace("Province, territory, outside of Canada:", "Province, territory, outside of Canada: Outside of Canada")
        .split(": ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None

In [66]:
us_charities = []
new_errors = []
for error in errors:
    details = parse_us_charity_details(text)
    if details:
        us_charities.append(details)
    else:
        new_errors.append(text)

In [60]:
len(reformatted)

202

In [39]:
print(reformatted)

[{'Charity name:': 'Église de Jésus-Christ: «Le bon berger»', 'BN/Registration number:': '837855501RR0001', 'Charity status:': 'Registered', 'Effective date of status: 2011-12-16': 'Address: 937 RUE ROUVILLE', 'City: REPENTIGNY': 'Province, territory, outside of Canada:', 'QUEBEC': 'Postal code/Zip code: J5Y1X9', 'Charity type:': 'Advancement of Religion', 'Category:': 'Christianity'}, {'Charity name:': 'A & O: Support Services for Older Adults Inc.', 'BN/Registration number:': '106688377RR0001', 'Charity status:': 'Registered', 'Effective date of status: 1969-01-01': 'Address: 200-207 DONALD STREET', 'City: WINNIPEG': 'Province, territory, outside of Canada:', 'MANITOBA': 'Postal code/Zip code: R3C1M5', 'Charity type:': 'Relief of Poverty', 'Category:': 'Organizations Relieving Poverty'}, {'Charity name:': 'A Way Home: Working Together to End Youth Homelessness', 'BN/Registration number:': '795393891RR0001', 'Charity status:': 'Registered', 'Effective date of status: 2022-01-01': 'Add

In [31]:
def parse_us_charity_details(detailed_text):
    details = {}
    formatted = (
        detailed_text.replace("Sanction: Note: this link will load in another window or tab", "")
        .replace("Charity status: Registered", "Charity status:  Registered")
        .replace("Province, territory, outside of Canada:", "Province, territory, outside of Canada: Outside of Canada")
        .split("  ")
    )
    listed = [item for item in formatted if item != ""]
    try:
        details = {
            listed[index]: listed[index + 1] for index in range(0, len(listed), 2)
        }
        return details
    except IndexError:
        return None


In [32]:
reformatted = []
us_errors = []
for text in errors:
    details = parse_us_charity_details(text)
    if details:
        reformatted.append(details)
    else:
        us_errors.append(text)

In [34]:
reformatted

[]

In [33]:
us_errors

['Charity name:  Acumen Canada  BN/Registration number:  818192130RR0001  Charity status: Registered  Effective date of status: 2013-07-16  Sanction: Note: this link will load in another window or tab  Address: 40 WORTH STREET  City: NEW YORK  Province, territory, outside of Canada:  Postal code/Zip code: 10013  Charity type:  Relief of Poverty  Category:  Organizations Relieving Poverty',
 'Charity name:  BLUE SKIES CULTURAL CENTRE  BN/Registration number:  130206220RR0001  Charity status: Registered  Effective date of status: 1994-01-01  Sanction: Note: this link will load in another window or tab  Address: 5141 CHESSIE CIRCLE  City: HALTOM CITY  Province, territory, outside of Canada:  Postal code/Zip code: 76137  Charity type:  Other purposes beneficial to the community  Category:  Arts',
 'Charity name:  Canadian Stress Society Stresshelp /La Société Canadienne du Stress Stressaide  BN/Registration number:  816792295RR0001  Charity status: Registered  Effective date of status: 201